# Object Size Detector

This object size detector application is one of a series of computer vision (CV) reference implementations using the Intel® Distribution of OpenVINO™ toolkit. This application demonstrates how to use CV to detect and measure the approximate length, width and size of assembly line parts. It is designed to work with an assembly line camera mounted above the assembly line belt. The application monitors mechanical parts as they are moving down the assembly line and raises an alert if it detects a part on the belt outside a specified size range.

https://github.com/intel-iot-devkit/object-size-detector-python

The implementation here (see <a href="ObjectSizeDetector.py">ObjectSizeDetector.py</a>) has been slightly modified to work in the IoT DevCloud environment.

### Demo

Select the cell below containing a Python code (to select, click with the mouse or use keyboard arrows). Then run the cell by pressing **Shift+Enter** or **CTRL+Enter** or by clicking the Run button in the toolbar.

A job is submitted to an edge compute nodes with Intel® Core CPU and Intel® Xeon CPU.
After the inference is completed, the output videos are appropriately stored in the /results/[device] directory, which can then be viewed within the Jupyter Notebook instance.

**Note**: The close button on the video will stop the video, but the application will continue to run. To stop the  application itself, interrupt the kernel. You can do this by going in the main menu to Kernel -> Interrupt.


## Set Up

### Import dependencies

Run the below cell to import Python dependencies needed for displaying the results in this notebook
(tip: select the cell and use **Ctrl+enter** to run the cell)

In [1]:
from pathlib import Path
import sys
sys.path.insert(0, str(Path().resolve().parent.parent))
from demoTools.demoutils import *

## Input Video

Run the following cell to create a symlink and view the input video.

In [2]:
!ln -sf ./resources/bolt-multi-size-detection.mp4 
videoHTML('Object size detector video', ['bolt-multi-size-detection.mp4'])

## Create a Job File

All the code up to this point has been run within the Jupyter Notebook instance running on a development node based on an Intel® Xeon® Scalable Processor, where the Notebook is allocated a single core. To run inference on the entire video, we need more compute power. We will run the workload on several DevCloud's edge compute nodes. We will send work to the edge compute nodes by submitting jobs into a queue. For each job, we will specify the type of the edge compute server that must be allocated for the job.

To pass the specific variables to the Python code, we will use following arguments:

* `-i`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;location of the input video
* `-o`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;output directory
* `-maxl`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;maximum length
* `-minl`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;minimum length
* `-maxw`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;maximum width
* `-minw`&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;minimum width

In [3]:
%%writefile object_size_detector.sh

cd $PBS_O_WORKDIR

VIDEO=$1
OUTPUT_DIR=$2
MAXLENGTH=$3
MINLENGTH=$4
MAXWIDTH=$5
MINWIDTH=$6

mkdir -p $OUTPUT_DIR

python3 ObjectSizeDetector.py -i ${VIDEO} \
                              -maxl ${MAXLENGTH} \
                              -minl ${MINLENGTH} \
                              -maxw ${MAXWIDTH} \
                              -minw ${MINWIDTH} \
                              -o    $OUTPUT_DIR

Writing object_size_detector.sh


In [4]:
!pbsnodes | grep compnode | awk '{print $3}' | sort | uniq -c

     35 idc001skl,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe
     15 idc002mx8,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-r,iei-mustang-v100-mx8
     17 idc003a10,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,hddl-f,iei-mustang-f100-a10
     23 idc004nc2,compnode,iei,tank-870,intel-core,i5-6500te,skylake,intel-hd-530,ram8gb,net1gbe,ncs,intel-ncs2
      6 idc006kbl,compnode,iei,tank-870,intel-core,i5-7500t,kaby-lake,intel-hd-630,ram8gb,net1gbe
     13 idc007xv5,compnode,iei,tank-870,intel-xeon,e3-1268l-v5,skylake,intel-hd-p530,ram32gb,net1gbe
     15 idc008u2g,compnode,up-squared,grove,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe,ncs,intel-ncs2
      1 idc009jkl,compnode,jwip,intel-core,i5-7500,kaby-lake,intel-hd-630,ram8gb,net1gbe
      1 idc010jal,compnode,jwip,intel-atom,e3950,apollo-lake,intel-hd-505,ram4gb,net1gbe
      1 idc011ark2250s,compnode,advantech,intel-

Here, the properties describe the node, and number on the left is the number of available nodes of that architecture.

##  Job queue submission

Each of the cells below will submit a job to different edge compute nodes.
The output of the cell is the `JobID` of your job, which you can use to track progress of a job.

**Note** You can submit all jobs at once or follow one at a time. 

After submission, they will go into a queue and run as soon as the requested compute resources become available. 
(tip: **shift+enter** will run the cell and automatically move you to the next cell. So you can hit **shift+enter** multiple times to quickly run multiple cells)

#### Submitting to an edge compute node with an Intel® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88186/Intel-Core-i5-6500TE-Processor-6M-Cache-up-to-3-30-GHz-">Intel® Core™ i5-6500TE processor</a>. The inference workload will run on the CPU.

In [5]:
#Submit job to the queue
job_id_core = !qsub object_size_detector.sh -l nodes=1:idc001skl:tank-870:i5-6500te -F "resources/bolt-multi-size-detection.mp4 results/core 250 228 45 28" -N object_size_core
print(job_id_core[0])
if job_id_core:
    progressIndicator('results/core', 'i_progress_'+job_id_core[0]+'.txt', "Inference", 0, 100)

4000.v-qsvr-1.devcloud-edge


#### Submitting to an edge compute node with Intel® Xeon® CPU
In the cell below, we submit a job to an <a 
    href="https://software.intel.com/en-us/iot/hardware/iei-tank-dev-kit-core">IEI 
    Tank* 870-Q170</a> edge node with an <a 
    href="https://ark.intel.com/products/88178/Intel-Xeon-Processor-E3-1268L-v5-8M-Cache-2-40-GHz-">Intel® 
    Xeon® Processor E3-1268L v5</a>. The inference workload will run on the CPU.

In [6]:
#Submit job to the queue
job_id_xeon = !qsub object_size_detector.sh  -l nodes=1:idc007xv5:e3-1268l-v5 -F "resources/bolt-multi-size-detection.mp4 results/xeon 250 228 45 28" -N object_size_xeon 
print(job_id_xeon[0]) 
#Progress indicators
if job_id_xeon:
    progressIndicator('results/xeon/', 'i_progress_'+job_id_xeon[0]+'.txt', "Inference", 0, 100)

4001.v-qsvr-1.devcloud-edge


### Check if the jobs are done

To check on the jobs that were submitted, use the `qstat` command.

We have created a custom Jupyter widget  to get live qstat update.
Run the following cell to bring it up.

In [7]:
liveQstat()

Output(layout=Layout(border='1px solid gray', height='300px', width='100%'))

Button(description='Stop', style=ButtonStyle())

You should see the jobs you have submitted (referenced by `Job ID` that gets displayed right after you submit the job).

There should also be an extra job in the queue "jupyterhub": this job runs your current Jupyter Notebook session.

The 'S' column shows the current status. 
- If it is in Q state, it is in the queue waiting for available resources. 
- If it is in R state, it is running. 
- If the job is no longer listed, it means it is completed.

**Note**: Time spent in the queue depends on the number of users accessing the edge nodes. Once these jobs begin to run, they should take from 1 to 5 minutes to complete.

## View Results

Once the jobs are completed, the queue system outputs the stdout and stderr streams of each job into files with names of the form

`object_size_{type}.o{JobID}`

`object_size_{type}.e{JobID}`

(here, object_size_{type} corresponds to the `-N` option of qsub).

However, for this case, we may be more interested in the output video files. They are stored in mp4 format inside the `results/[device]` directory.
We wrote a short utility script that will display these videos within the notebook.
Run the cells below to display them.
See `demoutils.py` if you are interested in understanding further on how the results are displayed in notebook.

In [8]:
videoHTML('IEI Tank (Intel Core CPU)', 
         ['results/core/inference_output.mp4'],'results/core/stats.txt')

In [ ]:
videoHTML('IEI Tank (Intel Xeon CPU)', 
         ['results/xeon/inference_output.mp4'],'results/xeon/stats.txt')